# Aeroelastic Multidisciplinary Analysis (MDA) Tutorial: Multi-Fidelity Version
# Panair--Panair
In this tutorial, we will see how to set up and perform a static aeroelastic analysis combining two fidelities using the [aerostructures](https://github.com/mid2SUPAERO/aerostructures) package (use the g.ruiz branch) and [OpenMDAO](https://github.com/mid2SUPAERO/OpenMDAO1).

The multifidelity of this version is introduced with __two different aerodynamic meshes__, while keeping the same fluid flow solver. This helps the user to identify easily the differences with the single fidelity version of the tutorial and makes it easier to transition to the Panair -- ADFlow version.

## Required files
Before actually running the Python code, we will see the files related to the the aerodynamic and structural analysis that are required to perform the MDA. These files should eb placed on the same folder as the main OpenMDAO script, which we will describe later.

### Structure model files
Two files are required for the structural model. The first one is 'nastran_static_template.inp'. This file looks mostly like a normal Nastran file but with some differences. In this file, instead of the classical Nastran FORCE cards, there are modified cards that include a dictionary key for each force component, for example:

`FORCE,201,33566,0,1.0,{Fx33566},{Fy33566},{Fz33566}`

Another difference with respect to a normal Nastran file is that the GRID cards, which contain the node coordinates, are also modified to include a dictionary key for each component of the nodal coordinates, for example:

`GRID,33566,,{x33566},{y33566},{z33566}`

The dictionary keys are used to create input files for Nastran in an automated way. As the nodal forces vary during the MDA, the input force values are updated by substituting the dictionary keys in the template by the contents of the dictionary. The same operation is performed for the GRID cards. Even if in the case of an MDA the nodal coordinates remain constant, this feature is useful for shape optimization, in which the nodal coordinates will change with each optimization iteration. We will see this in more detail on the tutorial on aeroelastic optimization.

The last difference with respect to an ordinary Nastran file is that, at the end of the file, there is a list of node IDs. The nodes in this list represent the nodes used for displacement interpolation. Typically, this list is a subset of the total nodes in the structural model, since not all nodes are required to perform the displacement interpolation. However, in the present example, this list contains all the nodes as it is a relatively small model. each node ID is precede by a '$' sign, so they appear as comments to Nastran. An example:

`$List of nodes belonging to the outer skin
$33566`

The file 'nastran_input_geometry.inp' is a typical Nastran input file containing all the GRID points of the jig shape of the structural model (i.e., the shape of the wing structure when no forces are applied onto it). For example, for one GRID point, this looks like:

`GRID,33566,,33.5157,0.0010402,2.554365`

Here is the jig shape structural mesh for the example in this tutorial:
<img src="structure_mesh.png">

### Aerodynamics model file
The aerodynamic mesh is written on the file 'aero_template.wgs'. The mesh is written in the [NASA](https://ntrs.nasa.gov/search.jsp?R=19850014069) Langley Wireframe Geometry Standard [(LaWGS)](http://www.pdas.com/lawgs.html). This file contains the geometry of the jig shape of the wing (i.e., the aerodynamic shape of the wing when no forces are applied onto it).

Here is the aerodynamic surface mesh (jig shape) for the example in this tutorial:
<img src="aero_mesh.png">

## OpenMDAO main file
### Preliminary data
Next we describe step by step the main OpenMDAO file 'nastran_panair_mda_cruise.py' to assemble all the components and perform the MDA itself. First, we import all the necessary modules from OpenMDAO and 'aerostructures':

In [1]:
from __future__ import print_function

import numpy as np

from openmdao.api import Problem, Group, IndepVarComp, ScipyGMRES, SqliteRecorder, view_model

from aerostructures import NastranStatic, DisplacementTransfer, Panair, LoadTransfer, Interpolation, StaticStructureProblemDimensions, StaticStructureProblemParams, AeroProblemDimensions, AeroProblemParams, NLGaussSeidel, Filter

Next, we start the main of the program and we set some values for the displacement interpolation method and the symmetry of the problem:

In [2]:
if __name__ == "__main__":

    #Interpolation function type and setup
    function_type = 'thin_plate'
    bias = (1.,100.,1.)

    #Symmetry plane index
    sym_plane_index = 2

    #case_name = 'alpha_c'
    case_name = 'alpha_low'
    case_name_h = 'alpha_high'

Here we have set the type of radial basis function to use for the displacement interpolation method through `function_type` as well as the modification of the Euclidean norm through the use of the `bias` parameter, as described by [Rendall and Allen](https://onlinelibrary.wiley.com/doi/abs/10.1002/nme.2219). By setting `sym_plane_index` to 2 we indicate the y is the normal axis to the symmetry plane (x = 1, y = 2, z = 3). The variables `case_name` & `case_name_h` are just a name for the low & high fidelity case.

Next we set the input parameters for the flow conditions, the reference quantities and the material properties:

In [3]:
    #Problem parameters
    Sw = 383.689555
    V = 252.16168
    rho_a = 0.38058496
    Mach = 0.85
    alpha = 1.340
    b = 58.7629
    c = 7.00532
    E = 6.89e10
    nu = 0.31
    rho_s = 2795.67
    n = 1.
    #Aerodynamic template files for both fidelities
    #Hi-Fi
    aero_template_l = 'aero_template_l.wgs'
    #Lo-Fi
    aero_template_h = 'aero_template_h.wgs'
    # Multi-fidelity options 'low', for low-fidelity; 'high', for high-fidelity; 'multi', for multi-fidelity
    fidelity = 'multi'
    # Iterations for the low-fidelity part in multi-fidelity mode
    it_l = 15

Then we create an instance of `StaticStructureProblemDimensions` to get the properties of the structural model: the number of nodes for the displacement interpolation `ns` and their list of IDs `node_id`, the total number of nodes `ns_all` and their IDs `node_id_all`, and the number of stress values extracted from the results `n_stress`. The variables `tn`, `mn`, and `sn` are the number of thicknesses, point masses, and sections of the model. In this case they are all set to 0 as these values are already set in the nastran template file and they will not be modified by OpenMDAO.

In [4]:
    structure_problem_dimensions = StaticStructureProblemDimensions()
    
    ns = structure_problem_dimensions.ns
    ns_all = structure_problem_dimensions.ns_all
    node_id = structure_problem_dimensions.node_id
    node_id_all = structure_problem_dimensions.node_id_all
    n_stress = structure_problem_dimensions.n_stress
    u = np.zeros((ns, 3))
    ul = np.zeros((ns, 3)) #Auxiliary variable to transfer the displacement field between fidelities
    tn = 0
    mn = 0
    sn = 0

Next, we create two instances of 'AeroProblemDimensions' to get information on the size (`na`) and topology (`network_info`) of the aerodynamic mesh for both the high and low fidelity cases:

In [5]:
    #Low fidelity instance -- aero_template_l.wgs
    aero_problem_dimensions = AeroProblemDimensions(aero_template_l)
    na = aero_problem_dimensions.na
    network_info = aero_problem_dimensions.network_info
    
    #High fidelity instance -- aero_template_h.wgs
    aero_problem_dimensions_h = AeroProblemDimensions(aero_template_h)
    na_h = aero_problem_dimensions_h.na
    network_info_h = aero_problem_dimensions_h.network_info

Now we create an instance of `StaticStructureProblemParams` to get the coordinates of the nodes used for displacement interpolation (`node_coord`) as well as the coordinates of all the nodes in the model (`node_coord_all`). Also, we get the coordinates of all the aerodynamic grid points `apoints_coord` for both fidelity levels:

In [6]:
    structure_problem_params = StaticStructureProblemParams(node_id, node_id_all)
    #Low fidelity instance -- aero_template_l.wgs 
    aero_problem_params = AeroProblemParams(aero_template_l)
    
    #High fidelity instance -- aero_template_h.wgs
    aero_problem_params_h = AeroProblemParams(aero_template_h)

    node_coord = structure_problem_params.node_coord
    node_coord_all = structure_problem_params.node_coord_all

    apoints_coord = aero_problem_params.apoints_coord
    apoints_coord_h = aero_problem_params_h.apoints_coord

### OpenMDAO model
Here we will create the OpenMDAO model itself, assembling all the components and creating the necessary groups. For more details on the use of OpenMDAO and to find introductory tutorials, refer to [OpenMDAO Tutorials](https://openmdao.readthedocs.io/en/1.7.3/usr-guide/tutorials.html).

First we define the top-level problem and we add to it all the independent variables of the problem (i.e., all the parameters):

In [7]:
    top = Problem()
    root = top.root = Group()

    #Add independent variables
    root.add('wing_area', IndepVarComp('Sw', Sw))
    root.add('airspeed', IndepVarComp('V', V))
    root.add('air_density', IndepVarComp('rho_a', rho_a))
    root.add('Mach_number', IndepVarComp('Mach', Mach))
    root.add('angle_of_attack', IndepVarComp('alpha', alpha))
    root.add('wing_span', IndepVarComp('b', b))
    root.add('wing_chord', IndepVarComp('c', c))
    root.add('s_coord', IndepVarComp('node_coord', node_coord))
    root.add('s_coord_all', IndepVarComp('node_coord_all', node_coord_all))
    root.add('a_coord', IndepVarComp('apoints_coord', apoints_coord))
    root.add('a_coord_h', IndepVarComp('apoints_coord', apoints_coord_h))
    root.add('load_factor', IndepVarComp('n', n))
    
    

Next, we create two mda groups, (`mda_l`) and (`mda_h`) that will contain all the disciplines of the aeroelastic MDA. The `DisplacementTransfer` component provides the displacements of the aerodynamic grid given the structural displacements, `aerodynamics` computes the aerodynamic forces from the current grid displacement and the airflow parameters. Then these forces are transferred to the aerodynamic mesh via the `load_transfer` and the `structures component` computes the displacements from the current aerodynamic forces. In the case of the high fidelity level, a `mult_filter` component is added to work as a switch from the low fidelity instance to the high fidelity MDA:

In [8]:
    #Lo-Fi Group
    mda_l = Group()

    #Add disciplines to the low fidelity group CHECK INPUTS
    mda_l.add('displacement_transfer', DisplacementTransfer(na, ns)) 
    mda_l.add('aerodynamics', Panair(na, network_info, case_name, aero_template_l, sym_plane_index=sym_plane_index)) 
    mda_l.add('load_transfer', LoadTransfer(na, ns))
    mda_l.add('structures', NastranStatic(node_id, node_id_all, n_stress, tn, mn, sn, case_name))
    
    
    #Hi-Fi Group
    mda_h = Group()
    
    #Add disciplines to the high-fidelity group CHECK INPUTS
    mda_h.add('mult_filter', Filter(ns, fidelity))
    mda_h.add('displacement_transfer_h', DisplacementTransfer(na_h, ns))
    mda_h.add('aerodynamics_h', Panair(na_h, network_info_h, case_name_h, aero_template_h, sym_plane_index=sym_plane_index))    
    mda_h.add('load_transfer_h', LoadTransfer(na_h, ns))
    mda_h.add('structures_h', NastranStatic(node_id, node_id_all, n_stress, tn, mn, sn, case_name_h))
    
    

Then we add an `Interpolation` component. This component gives, provided the structural and aerodynamic point coordinates, the interpolation matrix `H` as defined by [Rendall and Allen](https://onlinelibrary.wiley.com/doi/abs/10.1002/nme.2219), depending on the function type and the definition of the norm to be used. Since the size of the problem is different for both fidelity levels, we need to define one interpolation componen for each mda group.

In [9]:
    mda_l.add('inter', Interpolation(na, ns, function = function_type, bias = bias))
    mda_h.add('inter_h', Interpolation(na_h, ns, function = function_type, bias = bias))

Then we set the type of nonlinear solver for the MDA loop, in this case we use nonlinear Gauss--Seidel. As the convergence criterion, we set the relative error on the norm of the unknowns vector to be lower than `1.e-2`. We also use the [Aitken](https://en.wikipedia.org/wiki/Aitken%27s_delta-squared_process) convergence acceleration method with some recommended settings. Finally we add the `mda_group` to the top-level problem. This parameters allow the program to establish a condition for a fidelity switch (iterations, tolerances, etc.)

In [10]:
    #Define solver type and tolerance for MDA Lo-Fi
    mda_l.nl_solver = NLGaussSeidel()
    #The solver execution limit is used to control fidelity levels
    if fidelity == 'high':
        mda_l.nl_solver.options['maxiter'] = 0 #No Lo-Fi iterations
    elif fidelity == 'multi':
        mda_l.nl_solver.options['maxiter'] = it_l #Adds the limit for the execution of the MDA Solver
          
    mda_l.nl_solver.options['rutol'] = 1.e-2 
    mda_l.nl_solver.options['use_aitken'] = True
    mda_l.nl_solver.options['aitken_alpha_min'] = 0.1
    mda_l.nl_solver.options['aitken_alpha_max'] = 1.5

    mda_l.ln_solver = ScipyGMRES()

    #Define solver type and tolerance for MDA Hi-Fi
    mda_h.nl_solver = NLGaussSeidel()
    #The solver execution limit is used to control fidelity levels
    if fidelity == 'low':
        mda_h.nl_solver.options['maxiter'] = 0
        
    mda_h.nl_solver.options['rutol'] = 1.e-2
    mda_h.nl_solver.options['use_aitken'] = True
    mda_h.nl_solver.options['aitken_alpha_min'] = 0.1
    mda_h.nl_solver.options['aitken_alpha_max'] = 1.5

    mda_h.ln_solver = ScipyGMRES()

Both groups are added to the root problem with its variables promoted. However, a series of parameters and inputs variables need to be connected explicitly to avoid having duplicated data (i.e. two different aerodynamic load vectors named the same way). These connections happen at the mda level and at the root level.

In [11]:
    
    
    root.add('mda_group_l', mda_l, promotes=['*'])
    
    #Explicit connection Lo-Fi
    root.mda_group_l.connect('displacement_transfer.delta','aerodynamics.delta')
    root.mda_group_l.connect('inter.H','displacement_transfer.H')
    root.mda_group_l.connect('structures.u','displacement_transfer.u')
    root.mda_group_l.connect('aerodynamics.f_a','load_transfer.f_a')
    root.mda_group_l.connect('load_transfer.f_node','structures.f_node')
    root.mda_group_l.connect('inter.H','load_transfer.H')
    root.connect('a_coord.apoints_coord','inter.apoints_coord')
    root.connect('a_coord.apoints_coord', 'aerodynamics.apoints_coord')
    #Connect Indep Variables
    root.connect('wing_area.Sw', 'aerodynamics.Sw')
    root.connect('airspeed.V', 'aerodynamics.V')
    root.connect('air_density.rho_a', 'aerodynamics.rho_a')
    root.connect('Mach_number.Mach', 'aerodynamics.Mach')
    root.connect('angle_of_attack.alpha', 'aerodynamics.alpha')
    root.connect('wing_span.b', 'aerodynamics.b')
    root.connect('wing_chord.c', 'aerodynamics.c')
    root.connect('load_factor.n','structures.n')
    root.connect('s_coord.node_coord', 'inter.node_coord')
    root.connect('s_coord_all.node_coord_all', 'structures.node_coord_all')
      
    root.add('mda_group_h', mda_h, promotes=['*'])
    
    #Explicit connection Hi-Fi
    root.mda_group_h.connect('displacement_transfer_h.delta','aerodynamics_h.delta')
    root.mda_group_h.connect('inter_h.H','displacement_transfer_h.H')
    root.mda_group_h.connect('mult_filter.us','displacement_transfer_h.u')
    root.mda_group_h.connect('aerodynamics_h.f_a','load_transfer_h.f_a')
    root.mda_group_h.connect('load_transfer_h.f_node','structures_h.f_node')
    root.mda_group_h.connect('inter_h.H','load_transfer_h.H')
    root.mda_group_h.connect('structures_h.u','mult_filter.u')
    root.connect('a_coord_h.apoints_coord','inter_h.apoints_coord')
    root.connect('a_coord_h.apoints_coord', 'aerodynamics_h.apoints_coord')
    #Connect Indep Variables
    root.connect('wing_area.Sw', 'aerodynamics_h.Sw')
    root.connect('airspeed.V', 'aerodynamics_h.V')
    root.connect('air_density.rho_a', 'aerodynamics_h.rho_a')
    root.connect('Mach_number.Mach', 'aerodynamics_h.Mach')
    root.connect('angle_of_attack.alpha', 'aerodynamics_h.alpha')
    root.connect('wing_span.b', 'aerodynamics_h.b')
    root.connect('wing_chord.c', 'aerodynamics_h.c')
    root.connect('load_factor.n','structures_h.n')
    root.connect('s_coord.node_coord', 'inter_h.node_coord')
    root.connect('s_coord_all.node_coord_all', 'structures_h.node_coord_all')
    
    #Multifidelity explicit connections
    
    root.connect('structures.u', 'mult_filter.ul')

As we will do some post-processing on the multidisciplinary analysis on the MDA, we add a recorder to save the multidisciplinary variable values for all the iterations. By default, all the variables are stored. Since the whole displacement and force vectors are stored for each iteration, the recorder file could be really large for models with many nodes or aerodynamic grid points. To avoid that, we could specify to store only certain variables using `recorder.options['includes'] =` and specifying a list of variables names. For example, we could choose to store the displacements of certain nodes.

In [12]:
    #Recorder Lo-Fi
    recorder_l = SqliteRecorder('mda_l.sqlite3')
    recorder_l.options['record_metadata'] = False
    #Recorder Hi-Fi
    recorder_h = SqliteRecorder('mda_h.sqlite3')
    recorder_h.options['record_metadata'] = False
    # recorder.options['includes'] =
    top.root.mda_group_l.nl_solver.add_recorder(recorder_l)
    top.root.mda_group_h.nl_solver.add_recorder(recorder_h)

Finally, we set up the OpenMDAO problem and we run it:

In [13]:
    #Define solver type
    root.ln_solver = ScipyGMRES()

    top.setup()
    view_model(top)
    top.run()

d:\programas\aerostructures\aerostructures\data_transfer\rbf_poly_bias.py:101: RuntimeWarning: divide by zero encountered in log
  result = r**2 * log(r)
d:\programas\aerostructures\aerostructures\data_transfer\rbf_poly_bias.py:101: RuntimeWarning: invalid value encountered in multiply
  result = r**2 * log(r)


##############################################
Setup: Checking root problem for potential issues...

The following parameters have no associated unknowns:
structures.E
structures.Ix
structures.Iy
structures.a
structures.m
structures.nu
structures.rho_s
structures.s
structures.t
structures_h.E
structures_h.Ix
structures_h.Iy
structures_h.a
structures_h.m
structures_h.nu
structures_h.rho_s
structures_h.s
structures_h.t
Group 'mda_group_l' has the following cycles: [['displacement_transfer', 'aerodynamics', 'load_transfer', 'structures']]
Group 'mda_group_h' has the following cycles: [['displacement_transfer_h', 'aerodynamics_h', 'load_transfer_h', 'structures_h', 'mult_filter']]

The following params are connected to unknowns that are updated out of order, so their initial values may contain uninitialized unknown values: ['mda_group_h.displacement_transfer_h.u', 'mda_group_l.displacement_transfer.u']

Setup: Check of root problem complete.
##############################################



## Post-processing
After running the MDA we can post-process the data stored in the recorder file and the output files of the structural and aerodynamic solvers for the last iteration. For that, we first run the script `MDA_plot.py`:

In [ ]:
%run MDA_plot.py

This generates two plots: One shows the the norm of the unknowns vector as a function of the iterations (which is used as the convergence criterion for the MDA):
<img src="plot_unknowns_norm.svg">
The other one shows the vertical displacement with the highest absolute value for each iteration:
<img src="plot_wingtip_disp.svg">

Before post-processing the pressure distribution of the last iteration, we need to convert the last aerodynamic shape (in `.wgs`) format to the GMSH format (`.msh`). For that, we use the script `wgs_to_gmsh.py`, which will ask for the name of the corresponding `.wgs` file, in that case `./alpha_c/aero_current`:

In [ ]:
%run wgs_to_gmsh.py


This will generate the GMSH file `aero_current.msh`, which is required by the script `panair_post.py`, which will ask for the GMSH file, in that case `./alpha_c/aero_current`:

In [ ]:
%run ./alpha_low/panair_post.py


This will generate the GMSH file`aero_current_post.msh`, which contains the pressure distribution in addition to the aerodynamic grid:
<img src="cp_distribution.png">